In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/NELBHU64000.zip" -d "/content/"

In [ ]:
!rm -rf "/content/__MACOSX"

In [ ]:
try:
    import hugsvision
except:
    !pip install -q hugsvision
    import hugsvision

print(hugsvision.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 20.9 MB/s eta 0:00:00


/usr/local/lib/python3.9/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


0.75.5


In [ ]:
from hugsvision.dataio.VisionDataset import VisionDataset

train, test, id2label, label2id = VisionDataset.fromImageFolder(
	"/content/NELBHU64000",
	test_ratio   = 0.15,
	balanced     = False,
	augmentation = False,
)

Split Datasets...
Training Dataset Elements:  51136
+---------+-------------------------+-------------------+-------------+----------------------+-------------------+----------+---------------+-------------------+------------------------------------+----------------------+----------------+----------------+-----------------+---------+-------------+-----------------------------+------+---------------------------+----------------+--------------------------------+----------------+--------+----------------+---------------------+--------------+--------------------+------------+--------+----------+------------+---------+-----------------+---------+---------------+---------------------+-------+------------------+----------+-----------------------+-----------------------+-------------------+-------+----------------------------------+-------------------------------+----------------------+-----------------------------+----------------------------------+----------+----------------------------+----

In [ ]:
huggingface_model = "google/vit-base-patch32-384"

In [ ]:
other_model = "microsoft/beit-base-patch16-224-pt22k-ft22k"

In [ ]:
import torch
import gc

torch.cuda.empty_cache()

In [ ]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import AutoFeatureExtractor, SwinForImageClassification
from transformers import BeitFeatureExtractor, BeitForImageClassification

trainer = VisionClassifierTrainer(
	model_name   = "OurSkinModel",
	train      	 = train,
	test      	 = test,
	output_dir   = "./out/",
	max_epochs   = 5,
	batch_size   = 32, # On RTX 2080 Ti
  lr 		     = 2e-5,
	fp16	     = False,
	model = ViTForImageClassification.from_pretrained(
	    huggingface_model,
	    num_labels = len(label2id),
	    label2id   = label2id,
	    id2label   = id2label,
			ignore_mismatched_sizes = True
	),
	feature_extractor = ViTFeatureExtractor.from_pretrained(
		huggingface_model,
		ignore_mismatched_sizes = True
	),
)